In [1]:
import molmodmt as m3t
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
import openmmtools as mmtools
from pdbfixer import PDBFixer
import nglview as nv

/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


In [2]:
forcefield = app.ForceField('amber99sbildn.xml')

In [3]:
system_pdb=PDBFixer('MisL_Phyre.pdb')

In [4]:
system_pdb.topology

<Topology; 1 chains, 505 residues, 3911 atoms, 4008 bonds>

In [5]:
missing_residues     = system_pdb.findMissingResidues()
nonstandard_residues = system_pdb.findNonstandardResidues()
missing_atoms        = system_pdb.findMissingAtoms()

print('Missing residues:')
print(missing_residues)
print('NonStandard residues:')
print(nonstandard_residues)
print('Missing atoms:')
print(missing_atoms)

Missing residues:
None
NonStandard residues:
None
Missing atoms:
None


In [6]:
system_pdb.missingTerminals

{<Residue 504 (PHE) of chain 0>: ['OXT']}

In [7]:
system_pdb.addMissingAtoms()

In [8]:
system_pdb.addMissingHydrogens(pH=7.4)

In [9]:
m3t.view(system_pdb)

NGLWidget()

In [10]:
system = forcefield.createSystem(system_pdb.topology, constraints=app.HBonds)

In [11]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 0*unit.kelvin
pressure    = None

In [12]:
friction   = 1.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = mm.LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

In [13]:
simulation_time = 0.5*unit.nanoseconds
simulation_steps = round(simulation_time/step_size)

In [14]:
simulation_steps

250000

In [15]:
for ii in range(mm.Platform.getNumPlatforms()):
    platform_name  = mm.Platform.getPlatform(ii).getName()
    platform       = mm.Platform.getPlatformByName(platform_name)
    platform_speed = platform.getSpeed()
    print('Plataforma {} con velocidad {}'.format(platform_name,platform_speed))
    del(platform_name, platform, platform_speed)

Plataforma Reference con velocidad 1.0
Plataforma CPU con velocidad 10.0
Plataforma CUDA con velocidad 100.0
Plataforma OpenCL con velocidad 50.0


In [16]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [24]:
initial_positions = system_pdb.positions

In [17]:
harmonic_potential = '0.5*K*((x-xo)^2+(y-yo)^2+(z-zo)^2)'

In [18]:
harmonic_restraint_force = mm.CustomExternalForce(harmonic_potential)

In [19]:
harmonic_restraint_force.addGlobalParameter('K', 500.0 * unit.kilocalories_per_mole/unit.angstrom**2)
harmonic_restraint_force.addPerParticleParameter('x0')
harmonic_restraint_force.addPerParticleParameter('y0')
harmonic_restraint_force.addPerParticleParameter('z0')

2

In [20]:
restrained_atoms = [0]

In [21]:
simulation = app.Simulation(system_pdb.topology, system, integrator, platform, properties)

In [23]:
simulation.context.setPositions(initial_positions)
simulation.context.setVelocitiesToTemperature(0*unit.kelvin)

In [ ]:
simulation.step(10000)

In [ ]:
for index in restrained_atoms:
        parameters = sampler_state.positions[index,:].value_in_unit_system(unit.md_unit_system)
        restraint_force.addParticle(index, parameters)

Esto podía haber sido hecho de manera más económica haciendo uso de por ejemplo https://openmmtools.readthedocs.io/en/latest/forces.html
o https://openmmtools.readthedocs.io/en/latest/_modules/openmmtools/forcefactories.html

In [ ]:
https://github.com/uibcdf/Academia/blob/master/Din%C3%A1mica_Molecular/Pozo_Harm%C3%B3nico.ipynb
https://github.com/uibcdf/Academia/blob/master/Din%C3%A1mica_Molecular/MetEncefalina.ipynb
https://openmmtools.readthedocs.io/en/latest/_modules/openmmtools/forcefactories.html
http://www.maccallumlab.org/news/2015/1/23/testing
http://mdtraj.org/latest/examples/openmm.html
http://mdtraj.org/development/api/reporters.html
https://openmmtools.readthedocs.io/en/latest/gettingstarted.html